# 인공지능 작사가 만들기

#### Step 1. 데이터 준비
- 데이터 파일 읽기
- 문장 단위로 저장

#### Step 2. 데이터 전처리
- 작사가를 만들기 위한 데이터 정제
- 토큰화
- 평가 데이터셋 분리
- dataset 객체 생성

#### Step 3. 인공지능 만들기
- 모델 생성
- 훈련 (val_loss 2.2 이하)
- 결과 확인

## 데이터 준비

### 데이터 파일 읽기, 문장 단위로 저장

In [2]:
import glob
import os

# 데이터 파일 읽기
txt_file_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

# 문장 단위로 저장
raw_corpus = []
for txt in txt_list:
    with open(txt, "r") as f: 
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

# 결과 확인
print("data size:", len(raw_corpus))
print("examples:")
print(raw_corpus[0])
print(raw_corpus[1])
print(raw_corpus[2])

data size: 187088
examples:
Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?


## 데이터 전처리

### 작사가를 만들기 위한 데이터 정제

In [3]:
import re

# 토큰화를 위한 문장 정제
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 모두 소문자로
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence) # 마침표, 물음표, 느낌표, 쉼표 중복 제거
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence) # 소문자, 대문자, 마침표, 물음표, 느낌표 외 제거
    sentence = re.sub(r'[" "]+', " ", sentence) # 다중 공백 제거
    sentence = sentence.strip() # 문장 처음과 끝 공백 제거
    sentence = "<start> " + sentence + " <end>" # 문장 처음과 끝 <start> <end> 추가
    return sentence

In [4]:
# 정제된 문장 저장
corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0: continue # 빈 문장 제외
    
    preprocessed_sentence = preprocess_sentence(sentence) # 문장 정제 함수 실행
    if 15 < len(preprocessed_sentence.split()): continue # 토큰의 개수가 15개 초과하는 경우 제외
    
    corpus.append(preprocessed_sentence)

### 토큰화

In [5]:
import tensorflow as tf

# 토큰화
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 12000, # 단어 개수
        filters = ' ',
        oov_token = "<unk>" # 예외 단어
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus) # corpus(문장) -> tensor 변환
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding= 'post') # post(뒷 부분 pad) <-> pre(앞 부분 pad) 
    
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

In [6]:
# 결과 확인
for idx in tokenizer.index_word:
    print("{:<4} {:<4}".format(idx, tokenizer.index_word[idx]))
    if 10 <= idx: break

1    <unk>
2    <start>
3    <end>
4    i   
5    ,   
6    the 
7    you 
8    and 
9    a   
10   to  


### 평가 데이터셋 분리

In [7]:
from sklearn.model_selection import train_test_split

src = tensor[:, :-1] # <pad> 제외 (적은 가능성으로 <end>가 제외)
tgt = tensor[:, 1:] # <start> 제외

enc_train, enc_val, dec_train, dec_val = train_test_split(src, tgt, test_size = 0.2, random_state = 49)

# 결과 확인
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print(enc_val[0])
print(dec_val[0])

Source Train: (124981, 14)
Target Train: (124981, 14)
[ 2 49  5  3  0  0  0  0  0  0  0  0  0  0]
[49  5  3  0  0  0  0  0  0  0  0  0  0  0]


### dataset 객체 생성

In [33]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 128

# dataset 객체 생성
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder = True))
dataset

<BatchDataset shapes: ((128, 14), (128, 14)), types: (tf.int32, tf.int32)>

## 인공지능 만들기

### 모델 생성

In [34]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        # 레이어
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) # embedding layer (input, vector - id mapping)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences = True) # LSTM layer
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences = True) 
        self.linear = tf.keras.layers.Dense(vocab_size) # dense layer (output)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024 # word vector
hidden_size = 2048 # LSTM layer
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

### 훈련 (val_loss 2.2 이하)

In [35]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [36]:
model.compile(loss = loss, optimizer = optimizer)
model.fit(dataset, epochs = 10)

Epoch 1/10
976/976 [==============================] - 295s 284ms/step - loss: 3.0759
Epoch 2/10
976/976 [==============================] - 282s 289ms/step - loss: 2.5273
Epoch 3/10
976/976 [==============================] - 283s 289ms/step - loss: 2.1239
Epoch 4/10
976/976 [==============================] - 283s 289ms/step - loss: 1.7544
Epoch 5/10
976/976 [==============================] - 283s 290ms/step - loss: 1.4517
Epoch 6/10
976/976 [==============================] - 283s 290ms/step - loss: 1.2340
Epoch 7/10
976/976 [==============================] - 283s 290ms/step - loss: 1.0994
Epoch 8/10
976/976 [==============================] - 284s 291ms/step - loss: 1.0309
Epoch 9/10
976/976 [==============================] - 283s 290ms/step - loss: 1.0001
Epoch 10/10
976/976 [==============================] - 283s 290ms/step - loss: 0.9836


### 결과 확인

In [40]:
# 문장 생성 함수
def generate_text(model, tokenizer, init_sentence = "<start>", max_len = 14):
    # init_sentence -> tensor 변환
    init_sequence = tokenizer.texts_to_sequences([init_sentence])
    result_tensor = tf.convert_to_tensor(init_sequence, dtype = tf.int64)
    
    while True:
        predict = model(result_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis = -1), axis = -1)[:, -1]
        result_tensor = tf.concat([result_tensor, tf.expand_dims(predict_word, axis = 0)], axis = -1)
        
        if predict_word.numpy()[0] == tokenizer.word_index["<end>"]: break
        if result_tensor.shape[1] >= max_len: break
    
    text = ""
    for word_index in result_tensor[0].numpy():
        text += tokenizer.index_word[word_index] + " "
    
    return text

In [41]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you more than i love myself , <end> '

In [39]:
# test data 평가
score = model.evaluate(enc_val, dec_val, verbose = 1)

977/977 [==============================] - 39s 40ms/step - loss: 2.2219


# 회고

## 프로젝트 평가 및 한계점

- 결과물의 퀄리티<br/>
"i love you more than i love myself ,"<br/>
내가 얻은 문장은 쉼표로 끝나는 오류가 있다.<br/>
하지만 쉼표로 끝나는 것 외에는 문법적으로나 의미적으로 어색하지 않은 것 같다.<br/>
사랑 노래의 가사로도 나올 법한 문장인 것 같아 이 정도면 훌륭하지 않나 생각한다.<br/>
다만 실제로 이 모델을 사용한다고 가정하면 모델이 문장을 생성한 뒤 사람이 직접 평가를 해야할 필요가 있다.<br/>

- loss<br/>
test data로 평가한 loss가 2.2 수준으로 나와 node의 요구사항에 부합한다.<br/>
하지만 model.fit 단계에서 예측한 loss와 test data로 평가한 loss가 약 2.5배 차이나므로 이를 줄일 필요가 있다.<br/>
overfitting 방지하는 방법으로는 regularization strength를 올리는 방법이 있다.<br/>
시간이 부족하므로 현재는 수행하지 못했다.<br/>

- train/test 데이터 분리에 대한 의문<br/>
node에서는 토큰화를 실행한 뒤, train/test 데이터를 분리하라고 지시되어 있지만, 다음 글에서는 토큰화 후 데이터 분리에 대해 부정적인 의견을 표하고 있다.<br/>
아직 지식이 없어 어떤 상황에서, 어떤 이유에서 이러한 조언을 주는 지 파악하지 못해 아쉬움이 남는다.<br/>
<br/>
[stackoverflow, processing before or after train test split, (2022.01.18)](https://stackoverflow.com/questions/57693333/processing-before-or-after-train-test-split)
<br/>
[StackAbuse, Python for NLP: Multi-label Text Classification with Keras, Usman Malik, (2022.01.18)](https://stackabuse.com/python-for-nlp-multi-label-text-classification-with-keras/)
<br/>

## 학습 내용

- with open(...) as f<br/>
"with open A as B" 이 구문에서 A는 'context manager'가 되는 객체이고, B는 A enter 메서드의 리턴값이다.<br/>
즉, 위 구문의 f는 open(...) 함수가 리턴한 file object이고, f = open(...)라고 볼 수 있다.<br/>
<br/>
[Python)with open(...) as f에서 f의 정체는?, Jun-young Cha, (2020.02.22)](https://starriet.medium.com/python-with-open-as-f-%EC%97%90%EC%84%9C-f%EC%9D%98-%EC%A0%95%EC%B2%B4%EB%8A%94-3cb48ea9e302)
<br/>

- 정규표현식<br/>
언어를 원하는 형식으로 맞추기 위해서 정규표현식이 용이하다.<br/>
정규표현식은 다양한 활용 방안이 있는데, 이때 이용하기 좋은 사이트가 있다.<br/>
<br/>
[정규표현식](https://regexr.com/)
<br/>

- list append()와 extend()<br/>
list.append(iterable)은 리스트 끝에 iterable을 그대로 넣습니다.<br/>
list.extend(iterable)은 리스트 끝에 iterable의 모든 항목을 넣습니다.<br/>
<br/>
[Python- list append()와 extend()의 차이점, 네이버 블로그, apple, (2019.05.19)](https://m.blog.naver.com/wideeyed/221541104629)
<br/>

- LSTM(Long-Short-Term Memory)<br/>
이 모델은 LSTM layer를 사용하는데, LSTM은 RNN의 한 종류이다.<br/>
여러 문장의 어떤 키워드를 추측할 때, 뒷 맥락을 모두 알아야 한다면 즉 long-term dependencies를 다뤄야 한다면 기존 RNN 보다 LSTM을 사용하는 편이 좋다.<br/>
<br/>
[Long-Short-Term Memory (LSTM) 이해하기, Machine Learning, (2018.04.10)](https://dgkim5360.tistory.com/entry/understanding-long-short-term-memory-lstm-kr)
<br/>